In [ ]:
%load_ext autoreload
%autoreload 2

from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, OKXRestAPI, GateIORestAPI
from jaref_bot.utils.files import get_saved_coins

import polars as pl
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import json

from jaref_bot.strategies.funding import get_arbitrage_fund
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from jaref_bot.utils.coinmarketcap import get_top_tokens
# top_1000 = get_top_tokens(limit=1000, output_file='./data/top_1000_tokens.txt')

In [ ]:
saved_coins = get_saved_coins(data_folder='./data/agg_trades')

#### Исследование истории фандинга

In [ ]:
symbol = 'LPT'
exchange = 'bybit'

syms = set(['FIL', 'ONDO', 'GALA', 'ROSE', 'SNX', 'GTC', 'OP', 'CELO', 'APT', 'AKT', 'ARKM', 'STG', 'FLOW', 'C98',
       'ONG', 'PHA', 'SAND', 'ENJ', 'VET', 'CHR', 'OGN', 'MANA', 'ARB', 'GRT', 'VET', 'GMT', 'MANTA'])

exc_manager = ExchangeManager()
if exchange == 'bybit':
    exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
elif exchange == 'gate':
    exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

for symbol in syms:
    start_date = datetime.now().replace(microsecond=0) - timedelta(days=180)
    fund_hist = await exc_manager.get_funding_history(symbol=f'{symbol}_USDT', start_date=start_date, limit=500)
    fund_hist = fund_hist[0]
    fund_sum = float(fund_hist['funding'].sum()) * 100
    fmin = fund_hist['funding'].min() * 100
    fmax = fund_hist['funding'].max() * 100
    count = fund_hist['funding'].shape[0]
    
    print(f'Монета: {symbol}, Биржа: {exchange}')
    print(f'Первая запись: {fund_hist.index[0]}; Последняя запись: {fund_hist.index[-1]}')
    print(f'Суммарный фандинг: {fund_sum:.2f}%; Количество расчётов по фандингу: {count}')
    print(f'min: {fmin:.4f}%; max: {fmax:.4f}%')
    print()

In [ ]:
# Сохранение
# with open('./data/fund_coins.json', 'w') as f:
#     json.dump(coins, f, indent=4)  # indent для красоты (опционально)

In [ ]:
# Загрузка
with open('coins.json', 'r') as f:
    coins = json.load(f)

#### Текущий фандинг

In [ ]:
df = db_manager.get_table('funding_data')
df = pl.from_pandas(df)

In [ ]:
get_arbitrage_fund(df, 0.5)

In [ ]:
# Сильно отрицательный фандинг
df.filter(pl.col('funding_rate') < -1).sort(by='funding_rate')

In [ ]:
# Положительный фандинг (возможность для спот-фьючерс арбитража)
# df.filter(pl.col('funding_rate') > 0.1).sort(by='funding_rate', descending=True)

In [ ]:
# Частые выплаты (1 раз в 1-2 часа)
df.filter((pl.col('fund_interval').is_in([1, 2])) & (pl.col('funding_rate') < -0.15)).sort(by='funding_rate')

#### Расчёт одиночной сделки

In [ ]:
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI, OKXRestAPI
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("gate_spot", GateIORestAPI('spot'))


In [ ]:
interval = '1m'
n_iters = 1
sym = 'SKATE_USDT'

In [ ]:
res = await exc_manager.get_candles(symbol=sym, interval=interval, n_iters=n_iters)

In [ ]:
bb_linear = res['bybit_linear']
gt_spot = res['gate_spot']


In [ ]:
bb_linear

In [ ]:
df = linear_df[['Close']].merge(spot_df[['Close']], how='inner', on='Date', suffixes=(f'_linear', f'_spot'))
df['perc_diff'] = (df[f'Close_spot'] / df[f'Close_linear'] - 1) * 100
df['perc_diff'].iloc[:].plot(figsize=(14, 2));

In [ ]:
info = await exc_manager.get_prices()
index_price = info['bybit_linear'][sym]['index_price']
ask_price = info['bybit_linear'][sym]['ask_price']

print(f'Разница между индексной ценой и ценой фьючерса: {(index_price / ask_price - 1) * 100:.2f}%')

In [ ]:
# Сколько монет покупать
usdt_amount = 50
price = 0.1335

usdt_amount / price

#### Межбиржевой арбитраж

In [ ]:
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
# exc_manager.add_market("okx_linear", OKXRestAPI('linear'))
# exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

In [ ]:
interval = '1m'
n_iters = 1
sym = 'MAGIC_USDT'

res = await exc_manager.get_candles(symbol=sym, interval=interval, n_iters=n_iters)

In [ ]:
# exc_long = 'okx_linear'
exc_short = 'bybit_linear'

# long_df = res[exc_long]
short_df = res[exc_short]

In [ ]:
df = long_df[['Close']].merge(short_df[['Close']], how='inner', on='Date', suffixes=(f'_long', f'_short'))
df['perc_diff'] = (df[f'Close_short'] / df[f'Close_long'] - 1) * 100

fig, ax = plt.subplots(figsize=(14, 3))
df.index = df.index.tz_convert('Europe/Moscow')

ax.plot(df.index, df['perc_diff'])

ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()

In [ ]:
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)


In [ ]:
db_manager.update_data

In [ ]:
short_df

In [ ]:
edge_time = '2025-08-02 13:00'
orig_df = df[df.index < edge_time]
curr_df = df[df.index > edge_time]

minv, maxv, meanv, stdv = orig_df['perc_diff'].agg(['min', 'max', 'mean', 'std'])
cminv, cmaxv, cmeanv, cstdv = curr_df['perc_diff'].agg(['min', 'max', 'mean', 'std'])

curr_diff = curr_df['perc_diff'].iloc[-1]
# curr_dev = (curr_diff - meanv) / stdv

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 3), gridspec_kw={'width_ratios': [1, 1]})
ax1.set_title(f'Usual diff. mean: {meanv:.3f}, std: {stdv:.1f}')
ax1.plot(orig_df.index, orig_df['perc_diff'])
ax1.axhline(y=meanv, color='black', linestyle='-') # mean
ax1.axhline(y=meanv + 2 * stdv, color='b', linestyle='-'); # high bound
ax1.axhline(y=meanv - 2 * stdv, color='b', linestyle='-'); # low bound

ax2.set_title(f'Current diff. mean: {cmeanv:.3f}, std: {cstdv:.1f}, {curr_diff=:.3f}')
ax2.plot(curr_df.index, curr_df['perc_diff']);
ax2.axhline(y=cmeanv, color='black', linestyle='-') # mean
ax2.axhline(y=cmeanv + 2 * cstdv, color='b', linestyle='-'); # high bound
ax2.axhline(y=cmeanv - 2 * cstdv, color='b', linestyle='-'); # low bound

plt.tight_layout()
plt.show()

In [ ]:
def calculate_profit(qty: float, 
                     lm_open_price: float, 
                     lm_open_fee_perc: float, 
                     sm_open_price: float, 
                     sm_open_fee_perc: float,
                     lm_close_price: float,
                     lm_close_fee_perc: float, 
                     sm_close_price: float,
                     sm_close_fee_perc: float,
                    ):
    """
    Рассчитывает прибыль арбитражной сделки на основании цен открытия и закрытия, размера комиссии и фандинга.
    Diff всегда указывается как разница в процентах между курсом продажи и курсом покупки: 
        diff = (bid_price / ask_price - 1) * 100
    Args:
        qty - Количество купленных монет.
        lm_open_price - Ask price на лонг-рынке при открытии сделки.
        lm_open_fee_perc - Размер комиссии, указанный в процентах, при открытии сделки на лонг-рынке.
        sm_open_price - Bid price на шорт-рынке при открытии сделки.
        sm_open_fee_perc - Размер комиссии, указанный в процентах, при открытии сделки на шорт-рынке.
        lm_close_price - Bid price на лонг-рынке при закрытии сделки.
        lm_close_fee_perc - Размер комиссии, указанный в процентах, при закрытии сделки на лонг-рынке.
        sm_close_price - Ask price на шорт-рынке при закрытии сделки.
        sm_close_fee_perc - Размер комиссии, указанный в процентах, при закрытии сделки на шорт-рынке.

        
    """

    long_open_fee_perc = lm_open_fee_perc / 100
    short_open_fee_perc = sm_open_fee_perc / 100
    long_close_fee_perc = lm_close_fee_perc / 100
    short_close_fee_perc = sm_close_fee_perc / 100
    
    long_open_fee = qty * lm_open_price * long_open_fee_perc
    short_open_fee = qty * sm_open_price * short_open_fee_perc
    long_close_fee = qty * lm_close_price * long_close_fee_perc
    short_close_fee = qty * sm_close_price * short_close_fee_perc

    
    open_diff = (sm_open_price / lm_open_price - 1) * 100
    close_diff = (lm_close_price / sm_close_price - 1) * 100
    

In [ ]:
qty = 200
long_open_price = 0.4941
long_open_fee = qty * long_open_price * long_open_fee_perc
short_open_price = 0.4903
short_open_fee_perc = 0.1 / 100 # 0.012114 usdt, limit order
short_open_fee = qty * short_open_price * short_open_fee_perc
open_diff = (short_open_price / long_open_price - 1) * 100

In [ ]:
long_open_fee, short_open_fee

In [ ]:
long_close_price = 0.4760
long_close_fee_perc = 0.02 / 100
long_close_fee = qty * long_close_price * long_close_fee_perc
short_close_price = 0.4752
short_close_fee_perc = 0.036 / 100 # market order
short_close_fee = qty * short_close_price * short_close_fee_perc
close_diff = (long_close_price / short_close_price - 1) * 100


In [ ]:
close_diff

In [ ]:
long_close_fee, short_close_fee

In [ ]:
long_profit = qty * (long_close_price - long_open_price) - long_open_fee - long_close_fee
short_profit = qty * (short_open_price - short_close_price) - short_open_fee - short_close_fee
long_profit, short_profit

In [ ]:
long_close_fee * long_close_price, short_close_price * short_close_fee

In [ ]:
long_fund_perc = -1.246 / 100
long_funding = -qty * long_fund_perc * long_open_price

short_fund_perc = -0.202 / 100
short_funding = qty * short_fund_perc * short_open_price
funding_profit = long_funding + short_funding

In [ ]:
long_funding, short_funding

In [ ]:
print(f'Доход по лонг-рынку: {long_profit:.3f} {long_funding:.3f} (фандинг)')
print(f'Доход по шорт-рынку: {short_profit:.3f} {short_funding:.3f} (фандинг)')
print(f'Доход только по фандингу: {funding_profit:.3f}')
print(f'Итого: {long_profit + short_profit + funding_profit:.3f}')

#### Spot-futures arbitrage

In [ ]:
from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI, OKXRestAPI

import matplotlib.pyplot as plt

import pandas as pd
import polars as pl
import numpy as np
from datetime import datetime

exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("okx_spot", OKXRestAPI('linear'))

In [ ]:
interval = '1m'
n_iters = 1
sym1 = 'AIDOGE_USDT'
sym2 = '10000000AIDOGE_USDT'

bb_lin = await exc_manager.get_candles(symbol=sym2, interval=interval, n_iters=n_iters)
okx_lin = await exc_manager.get_candles(symbol=sym1, interval=interval, n_iters=n_iters)

In [ ]:
bb_lin = bb_lin[0]
okx_lin = okx_lin[1]

In [ ]:
okx_lin[['High', 'Low', 'Close']] = okx_lin[['High', 'Low', 'Close']] * 10_000_000

In [ ]:
cols = ['High', 'Low', 'Close']
df = okx_lin[cols].merge(bb_lin[cols], on='Date', suffixes=('_okx', '_bb'))
df['close_diff'] = (df['Close_bb'] / df['Close_okx'] - 1) * 100
# df['max_diff'] = (df['High_spot'] / df['Low_fut'] - 1) * 100
# df['min_diff'] = (df['Low_spot'] / df['High_fut'] - 1) * 100

In [ ]:
df[['close_diff']].plot(figsize=(14, 4));
plt.axhline(y=df['close_diff'].median(), color='black', linestyle='-') # mean
plt.axhline(y=0, color='red', linestyle='-'); # mean

In [ ]:
df.iloc[-1]

In [ ]:
# Цена входа в сделку
short_price = 0.0028212
long_price = 0.002804
long_price, short_price

In [ ]:
(short_price / long_price - 1) * 100

In [ ]:
# Выход из сделки
short_close_price = df.iloc[-1]['Close_bb']
long_close_price = df.iloc[-1]['Close_okx']
(short_close_price / long_close_price - 1) * 100